In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Kaggle Workshops Starter Notebook [Python version]
## Online Retail Store: Predicting Visitor Behaviour 

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?name=Online%20Retail%20Store%3A%20Predicting%20Visitor%20Behaviour%20-%20Starter%20Notebook%20%5BPython%5D&download_url=https%3A%2F%2Fstorage.googleapis.com%2Fstarter_notebooks%2Fgoogle_analytics_sample_bqml_python.ipynb&url=https%3A%2F%2Fgist.github.com%2Fpolong-lin%2F51bb01bac82513294e10fb9856c4fdb0">
      <img src="https://cloud.google.com/images/products/ai/ai-solutions-icon.svg" alt="AI Platform Notebooks">Run on AI Platform Notebooks</a>
  </td>
</table>

- [Setup BigQuery in Jupyter](#setup)
- [BigQuery ML basics](#basics)
  - [Select BigQuery data from Python](#select)
  - [Train a BigQuery ML model](#train)
  - [Check models currently being trained on BigQuery](#currentlytraining)
  - [Check if a model has finished training](#finishedtraining)
  - [Evaluate a BigQuery ML model](#evaluate)
  - [Compare multiple models](#compare)
  - [ML.FEATURE_INFO to check input feature information](#featinfo)
  - [Make predictions using BigQuery ML](#predictions)
  - [Export predictions as a CSV file](#exportcsv)
- [Feature engineering](#feateng)
  - [EXTRACT: Extract dates and time](#extractdates)
  - [REGEXP_EXTRACT_ALL: Extract words from a STRING column](#extractwords)
  - [ML.QUANTILE_BUCKETIZE: Bucketize your numeric feature into bins](#bucketize)
  - [ML.POLYNOMIAL_EXPAND: Combine numeric features](#polynomialexpand)
  - [ML.FEATURE_CROSS: Combine categorical features](#featurecross)
- [Feature selection](#featselection)
  - [ML.FEATURE_IMPORTANCE for Boosted Tree models](#featimportance)

<a id="setup"></a>

### Setup BigQuery in Jupyter 

#### Set parameters

In [3]:
#Check current GCP project ID
!gcloud config get-value project

polong-sandbox


In [1]:
PROJECT_ID = "MY-PROJECT-ID" #REPLACE with project ID
DATASET_ID = "bqmlretail"
LOCATION = "EU"

#### Create dataset if it does not yet exist

In [ ]:
!bq mk --location=$LOCATION $DATASET_ID

#### Required libraries

In [9]:
#!pip install google-cloud-bigquery --upgrade --quiet

In [2]:
from google.cloud import bigquery
from google.api_core.exceptions import BadRequest
import time
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

# Construct a BigQuery client object.
client = bigquery.Client()

def bq_query(sql, async_flag=False):
    """
    If `async_flag` set to False (default), 
        returns the query results for `sql` as a Pandas DataFrame, 
    Else, submits query asynchronously and returns nothing.
    """
    
    # Try dry run before executing query to catch any errors
    try:
        job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
        dry_run_job = client.query(sql, job_config=job_config)
    except BadRequest as err:
        print(err)
        return
        
    job_config = bigquery.QueryJobConfig()
    df = client.query(sql, job_config=job_config)
    
    if async_flag:
        return f"Query sent asynchronously for job id: {df.job_id}"
    else:
        df = df.result() #wait for query to finish running
        return df.to_dataframe()
    
    
print("Done.")

Done.


<a id="basics"></a>

## BigQuery ML basics

<a id="select"></a>

#### Select BigQuery data from Python

In [97]:
sql_select = """
SELECT 
  *
FROM
  kaggleworkshops.google_analytics_sample_eu.train
WHERE date BETWEEN TIMESTAMP("2016-08-01")
  AND TIMESTAMP("2016-08-31")
"""

df = bq_query(sql = sql_select)

df.head()

,fullVisitorId,visitStartTime,date,deviceCategory,isMobile,operatingSystem,browser,country,city,trafficSource,trafficMedium,trafficCampaign,isFirstVisit,isBounce,totalVisits,totalHits,totalPageviews,totalTimeOnSite,totalTransactions,productPagesViewed,addedToCart
0,4283942788674999975,1470152213,2016-08-02 00:00:00+00:00,mobile,True,iOS,Safari,United Kingdom,not available in demo dataset,(direct),(none),(not set),0,0,1,24,17,386,0,7,0
1,690675959644230293,1470175915,2016-08-02 00:00:00+00:00,mobile,True,iOS,Safari,United States,San Francisco,google,organic,(not set),1,0,1,25,22,186,0,1,0
2,0952928557794452561,1470190050,2016-08-02 00:00:00+00:00,tablet,True,iOS,Safari,Canada,Toronto,google,organic,(not set),1,0,1,42,38,540,0,0,0
3,0275527783446979951,1470178107,2016-08-02 00:00:00+00:00,mobile,True,iOS,Safari,United States,not available in demo dataset,(direct),(none),(not set),0,0,1,40,35,797,0,1,0
4,503376809210729712,1470138468,2016-08-02 00:00:00+00:00,mobile,True,iOS,Safari,Singapore,Singapore,(direct),(none),(not set),1,0,1,60,49,984,0,8,0


<a id="train"></a>

#### Train a BigQuery ML model

In [100]:
sql_logreg_sample = """
CREATE OR REPLACE MODEL bqmlretail.logreg_sample
TRANSFORM(
  LOWER(country) country_lower,
  totalPageviews,
  addedToCart
)
OPTIONS(
  model_type='logistic_reg', 
  INPUT_LABEL_COLS=["addedToCart"]) AS
SELECT 
  *
FROM
  kaggleworkshops.google_analytics_sample_eu.train
WHERE date BETWEEN TIMESTAMP("2016-08-01")
  AND TIMESTAMP("2016-08-31")
"""

#send asynchronously
bq_query(sql = sql_logreg_sample,
        async_flag=True)

'Query sent asynchronously for job id: b72a3a3c-376f-4169-80f7-5810aeabf9c1'

Other model types for classification:
 - [`logistic_reg`](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create)
 - [`boosted_tree_classifier`](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-boosted-tree)
 - [`dnn_classifier`](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-dnn-models)
 - [`automl_classifier`](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-automl)

<a id="currentlytraining"></a>

#### Check models currently being trained on BigQuery

This will check if the models in your project are still undergoing training.

In [102]:
sql_currentlyrunningqueries = f"""
SELECT
    job_id,
    creation_time,
    query,
    REGEXP_EXTRACT(query, r'{DATASET_ID}\.\w+') AS modelname
FROM `region-eu`.INFORMATION_SCHEMA.JOBS_BY_USER
WHERE 
    REGEXP_CONTAINS(LOWER(query), 'create (or replace ){{0,1}}model')
    AND NOT REGEXP_CONTAINS(LOWER(query), 'information_schema')
    AND state != "DONE"
"""

print(sql_currentlyrunningqueries)

#send asynchronously
models_still_training = bq_query(sql_currentlyrunningqueries)

print(f"{len(models_still_training)} models still currently training:")
models_still_training


SELECT
    job_id,
    creation_time,
    query,
    REGEXP_EXTRACT(query, r'bqmlretail\.\w+') AS modelname
FROM `region-eu`.INFORMATION_SCHEMA.JOBS_BY_USER
WHERE 
    REGEXP_CONTAINS(LOWER(query), 'create (or replace ){0,1}model')
    AND NOT REGEXP_CONTAINS(LOWER(query), 'information_schema')
    AND state != "DONE"

0 models still currently training:


,job_id,creation_time,query,modelname


<a id="finishedtraining"></a>

#### Check which models have finished training

In [15]:
sql_finished_queries = f"""
SELECT
    job_id,
    creation_time,
    query,
    REGEXP_EXTRACT(query, r'{DATASET_ID}\.\w+') AS modelname
FROM `region-eu`.INFORMATION_SCHEMA.JOBS_BY_USER
WHERE 
    REGEXP_CONTAINS(LOWER(query), 'create (or replace ){{0,1}}model')
    AND NOT REGEXP_CONTAINS(LOWER(query), 'information_schema')
    AND state = "DONE"
ORDER BY creation_time DESC
"""

print(sql_finished_queries)

finished_models = bq_query(sql_finished_queries)
finished_models.head()


SELECT
    job_id,
    creation_time,
    query,
    REGEXP_EXTRACT(query, r'bqmlretail\.\w+') AS modelname
FROM `region-eu`.INFORMATION_SCHEMA.JOBS_BY_USER
WHERE 
    REGEXP_CONTAINS(LOWER(query), 'create (or replace ){0,1}model')
    AND NOT REGEXP_CONTAINS(LOWER(query), 'information_schema')
    AND state = "DONE"
ORDER BY creation_time DESC



,job_id,creation_time,query,modelname
0,b72a3a3c-376f-4169-80f7-5810aeabf9c1,2020-10-15 10:44:43.264000+00:00,"\nCREATE OR REPLACE MODEL bqmlretail.logreg_sample\nTRANSFORM(\n LOWER(country) country_lower,\n totalPageviews,\n addedToCart\n)\nOPTIONS(\n model_type='logistic_reg', \n INPUT_LABEL_COLS=[""...",bqmlretail.logreg_sample
1,09f0a1a2-2789-4995-b14c-e7000f311aec,2020-10-15 10:44:30.118000+00:00,"\nCREATE OR REPLACE MODEL bqmlretail.logreg_sample\nTRANSFORM(\n LOWER(country) country_lower,\n totalPageviews,\n addedToCart\n)\nOPTIONS(\n model_type='logistic_reg', \n INPUT_LABEL_COLS=[""...",bqmlretail.logreg_sample
2,1a1d2489-a32c-4b0f-b5dd-558abedaa991,2020-10-15 09:52:06.220000+00:00,"\nCREATE OR REPLACE MODEL bqmlretail.xgb_test12\nTRANSFORM(\n LOWER(country) country_lower,\n totalPageviews,\n addedToCart\n)\nOPTIONS(\n model_type='boosted_tree_classifier', \n INPUT_LABEL...",bqmlretail.xgb_test12
3,1685e478-499e-4bcf-921c-f61f66cf4ab8,2020-10-15 09:52:04.292000+00:00,"\nCREATE OR REPLACE MODEL bqmlretail.xgb_test11\nTRANSFORM(\n LOWER(country) country_lower,\n totalPageviews,\n addedToCart\n)\nOPTIONS(\n model_type='boosted_tree_classifier', \n INPUT_LABEL...",bqmlretail.xgb_test11
4,4b9aae28-3bee-45a6-a0df-1b1a6d2abf9a,2020-10-15 09:52:02.168000+00:00,"\nCREATE OR REPLACE MODEL bqmlretail.xgb_test10\nTRANSFORM(\n LOWER(country) country_lower,\n totalPageviews,\n addedToCart\n)\nOPTIONS(\n model_type='boosted_tree_classifier', \n INPUT_LABEL...",bqmlretail.xgb_test10


<a id="evaluate"></a>

#### Evaluate a BigQuery ML model

In [32]:
MODEL_NAME = "logreg_sample"

sql_logreg_sample = f"""
SELECT 
  *
FROM
  ML.EVALUATE(
      MODEL {DATASET_ID}.{MODEL_NAME})
"""

print(sql_logreg_sample)

bq_query(sql = sql_logreg_sample)


SELECT 
  *
FROM
  ML.EVALUATE(
      MODEL bqmlretail.logreg_sample)



,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.716418,0.326975,0.941029,0.449018,0.166795,0.929027


<a id="compare"></a>

### Compare multiple models

In [33]:
MODEL_LIST = ["logreg_sample", 
              "logreg_sample", 
              "logreg_sample"] #change or add models here

sql_modeleval_list = [f"SELECT \n  '{modelname}' as modelname, \n  * \nFROM \n  ML.EVALUATE(MODEL {DATASET_ID}.{modelname})"
                      for modelname 
                      in MODEL_LIST]

sql_compare_evaluation = "\n\nUNION ALL\n\n".join(sql_modeleval_list)

print(sql_compare_evaluation)

bq_query(sql_compare_evaluation)

SELECT 
  'logreg_sample' as modelname, 
  * 
FROM 
  ML.EVALUATE(MODEL bqmlretail.logreg_sample)

UNION ALL

SELECT 
  'logreg_sample' as modelname, 
  * 
FROM 
  ML.EVALUATE(MODEL bqmlretail.logreg_sample)

UNION ALL

SELECT 
  'logreg_sample' as modelname, 
  * 
FROM 
  ML.EVALUATE(MODEL bqmlretail.logreg_sample)


,modelname,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,logreg_sample,0.716418,0.326975,0.941029,0.449018,0.166795,0.929027
1,logreg_sample,0.716418,0.326975,0.941029,0.449018,0.166795,0.929027
2,logreg_sample,0.716418,0.326975,0.941029,0.449018,0.166795,0.929027


<a id="featinfo"></a>

### ML.FEATURE_INFO to check input feature information

The ML.FEATURE_INFO function allows you to see information about the input features used to train a model. [ML.FEATURE_INFO Documentation](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-feature)

In [8]:
MODEL_NAME = "logreg_sample"

sql_featureinfo = f"""
SELECT
    *
FROM
    ML.FEATURE_INFO(MODEL bqmlretail.{MODEL_NAME})
"""

bq_query(sql_featureinfo)

,input,min,max,mean,median,stddev,category_count,null_count
0,country,NaN,NaN,NaN,NaN,NaN,177.0,0
1,totalPageviews,0.0,469.0,4.850211,2.0,10.29755,NaN,0


<a id="predictions"></a>

### Make predictions using BigQuery ML

In [ ]:
MODEL_NAME = "logreg_sample"

sql_predict = f"""
SELECT
  sessionId, 
  prob as addedToCart
FROM
  ML.PREDICT(MODEL {DATASET_ID}.{MODEL_NAME},
    (
    SELECT 
      CONCAT(fullVisitorId, CAST(visitStartTime as string)) as sessionId, 
      *
    FROM
    `kaggleworkshops.google_analytics_sample_eu.test`
  )),
UNNEST(predicted_addedToCart_probs)
WHERE label = 1
"""

print(sql_predict)

submission = bq_query(sql_predict)
submission.head()

<a id="exportcsv"></a>

### Export predictions for Kaggle as a CSV file

In [ ]:
MODEL_NAME = "logreg_sample"

#save as CSV
submission.to_csv(f"submission_{MODEL_NAME}.csv", index=False)

The predictions are now saved to a CSV file. To download the file to your computer, locate your file in the left-menu in JupyterLab, right-click on it and click "Download". You can now make a submission to the Kaggle competition with the file.

<hr><a id="feateng"></a>

## Preprocessing Functions in BigQuery ML
You can use [TRANSFORM()](https://cloud.google.com/bigquery-ml/docs/bigqueryml-transform) with any of the [preprocessing functions](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-preprocessing-functions) below.

<a id="extractdates"></a>

#### EXTRACT: Extract dates and time

[EXTRACT from DATE documentation](https://cloud.google.com/bigquery/docs/reference/standard-sql/date_functions#extract)  
[EXTRACT from TIMESTAMP documentation](https://cloud.google.com/bigquery/docs/reference/standard-sql/timestamp_functions#extract)

In [10]:
sql_datetime = """
    SELECT
        date,
        EXTRACT( month FROM date) as month,
        EXTRACT( dayofweek FROM date) AS dayofweek,
        EXTRACT( day FROM date) as day,
        
        visitStartTime,
        EXTRACT( hour FROM TIMESTAMP_SECONDS(visitStartTime) ) as hour
    FROM
        kaggleworkshops.google_analytics_sample_eu.train
    LIMIT 10
    """

bq_query(sql_datetime, 
         async_flag = False)

,date,month,dayofweek,day,visitStartTime,hour
0,2016-10-14 00:00:00+00:00,10,6,14,1476492957,0
1,2017-04-30 00:00:00+00:00,4,1,30,1493611205,4
2,2017-03-20 00:00:00+00:00,3,2,20,1490071147,4
3,2017-04-01 00:00:00+00:00,4,7,1,1491048053,12
4,2017-05-20 00:00:00+00:00,5,7,20,1495292376,14
5,2017-05-14 00:00:00+00:00,5,1,14,1494792010,20
6,2017-05-17 00:00:00+00:00,5,4,17,1495047452,18
7,2017-05-15 00:00:00+00:00,5,2,15,1494883945,21
8,2016-10-11 00:00:00+00:00,10,3,11,1476231674,0
9,2016-10-10 00:00:00+00:00,10,2,10,1476144607,0


<a id="extractwords"></a>

#### REGEXP_EXTRACT_ALL: Extract words from a STRING column
[REGEXP_EXTRACT_ALL Documentation](https://cloud.google.com/bigquery/docs/reference/standard-sql/string_functions#regexp_extract_all)

In [14]:
sql_words = """
SELECT 
  trafficCampaign, 
  REGEXP_EXTRACT_ALL(trafficCampaign, 
                      r"[a-zA-Z]+"
                    ) as extracted_words 

FROM 
  kaggleworkshops.google_analytics_sample_eu.train
LIMIT 5
"""

bq_query(sql_words)

,trafficCampaign,extracted_words
0,(not set),"[not, set]"
1,(not set),"[not, set]"
2,(not set),"[not, set]"
3,(not set),"[not, set]"
4,(not set),"[not, set]"


<a id="bucketize"></a>

#### ML.QUANTILE_BUCKETIZE: Bucketize your numeric feature into bins
[ML.QUANTILE_BUCKETIZE Documentation](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-preprocessing-functions#quantile_bucketize)

In [12]:
sql_bucketize = """
SELECT 
    totalTimeOnSite,
    ML.QUANTILE_BUCKETIZE(totalTimeOnSite,
                            100 #number of buckets
                         ) OVER() AS bucket_totalTimeOnSite,
FROM
    kaggleworkshops.google_analytics_sample_eu.train
LIMIT 10
"""

bq_query(sql_bucketize)

,totalTimeOnSite,bucket_totalTimeOnSite
0,0,bin_1
1,0,bin_1
2,0,bin_1
3,302,bin_41
4,0,bin_1
5,0,bin_1
6,183,bin_36
7,7,bin_5
8,543,bin_45
9,164,bin_35


<a id="polynomialexpand"></a>

#### ML.POLYNOMIAL_EXPAND: Combine numeric features

[ML.POLYNOMIAL_EXPAND Documentation](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-preprocessing-functions#mlpolynomial_expand)

In [40]:
sql_polynomialexpand = """
SELECT
    ML.POLYNOMIAL_EXPAND(STRUCT(totalHits, totalPageViews), 
                          2) AS x
FROM
    kaggleworkshops.google_analytics_sample_eu.train
LIMIT 3
"""

bq_query(sql_polynomialexpand)

,x
0,"{'totalHits': 27.0, 'totalHits_totalHits': 729.0, 'totalHits_totalPageViews': 621.0, 'totalPageViews': 23.0, 'totalPageViews_totalPageViews': 529.0}"
1,"{'totalHits': 23.0, 'totalHits_totalHits': 529.0, 'totalHits_totalPageViews': 414.0, 'totalPageViews': 18.0, 'totalPageViews_totalPageViews': 324.0}"
2,"{'totalHits': 23.0, 'totalHits_totalHits': 529.0, 'totalHits_totalPageViews': 368.0, 'totalPageViews': 16.0, 'totalPageViews_totalPageViews': 256.0}"


<a id="featurecross"></a>

#### ML.FEATURE_CROSS: Combine categorical features

[ML.FEATURE_CROSS Documentation](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-preprocessing-functions#mlfeature_cross)

In [39]:
sql_featurecross = """
SELECT
    country,
    deviceCategory,
    ML.FEATURE_CROSS(STRUCT(country, deviceCategory), 
                    2) AS x
FROM
    kaggleworkshops.google_analytics_sample_eu.train
LIMIT 3
"""

bq_query(sql_featurecross)

,country,deviceCategory,x
0,United Kingdom,mobile,{'country_deviceCategory': 'United Kingdom_mobile'}
1,Taiwan,mobile,{'country_deviceCategory': 'Taiwan_mobile'}
2,Taiwan,mobile,{'country_deviceCategory': 'Taiwan_mobile'}


<a id="featselection"></a>
<hr>

## Feature Selection

<a id="featimportance"></a>

### ML.FEATURE_IMPORTANCE for Boosted Tree models

_Note: This function is only available for Boosted Tree models (`boosted_tree_classifier` or `boosted_tree_regressor`)._

The ML.FEATURE_IMPORTANCE function allows you to see feature importance score, which indicates how useful or valuable each feature was in the construction of the Boosted Tree model during training. [ML.FEATURE_IMPORTANCE Documentation](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-importance#mlfeature_importance_function)

In [ ]:
MODEL_NAME = "MY_BOOSTED_TREE_MODEL_NAME"

sql_featureimportance = f"""
SELECT
    *
FROM
    ML.FEATURE_IMPORTANCE(MODEL bqmlretail.{MODEL_NAME})
"""

bq_query(sql_featureimportance)